# Clip Tokenizer : Prompt/Caption Checker

Given a prompt or caption, it will tokenize it and display which tokens fall within 77 token limit and outside

https://github.com/yushan777/cliptokenizer

Run each cell in order.  



In [ ]:
#@title 1. Install Requirements

print("This will take a couple of minutes...")
!pip install -q --upgrade diffusers[torch]
!pip install -q transformers
print("Finished.")

In [3]:
#@title 2. Select Model

MODEL = 'SD1.5' # @param ['SD1.5', 'SD2.1', 'SDXL']

# @markdown SD1.5, SD2.1 and SDXL all use the same dictionaries. So it really doesn't matter which one you choose.<br />
# @markdown SDXL has two dictionaries, but they are also the same


In [ ]:
#@title 3. Setup Pipeline

import os
import torch
from transformers import CLIPTokenizer
from transformers import file_utils
from diffusers import StableDiffusionPipeline, DDIMScheduler

print("This will take a couple of minutes...")
# =======================================
# setup pipeline
if torch.cuda.is_available():
    torch.cuda.empty_cache()

MODEL_PATH = ''
if MODEL == 'SD1.5':
  MODEL_PATH = 'runwayml/stable-diffusion-v1-5'
elif MODEL == 'SD2.1':
  MODEL_PATH = 'stabilityai/stable-diffusion-2-1'
elif MODEL == 'SDXL':
  MODEL_PATH = 'stabilityai/stable-diffusion-xl-base-1.0'

    # MODEL_PATH = 'stabilityai/stable-diffusion-xl-base-1.0'
# MODEL_PATH = "runwayml/stable-diffusion-v1-5"
# sdxl has 2 tokenizers and two dictionaries, but they are the same


pipe = StableDiffusionPipeline.from_pretrained(f'{MODEL_PATH}', variant="fp16", torch_dtype = torch.float16)
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")

tokenizer = CLIPTokenizer.from_pretrained(MODEL_PATH, subfolder="tokenizer",variant="fp16")

# tokenizer/vocab.json files are normally saved in cache e.g.
# /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-v1-5/snapshots/xxxxx/tokenizer/vocab.json

# transformers cache directory
cache_dir = file_utils.default_cache_path

# find the vocab file
print("locating cached vocab.json")
for root, dirs, files in os.walk(cache_dir):
    for file in files:
        if file == "vocab.json":
            vocab_file_path = os.path.join(root, file)
            print(f"Found vocab file at: {vocab_file_path}")
            break

print("Finished.")


In [5]:
#@title 4. Setup Tokenize() Function

import json
import textwrap
# ===========================================================================
# for coloring text
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    RED = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

# ===========================================================================

def tokenize(prompt_or_caption, show_end_of_word_marker):

  with open(vocab_file_path, "r", encoding='utf-8') as f:
    vocab = json.load(f)
    vocab = {v: k for k, v in vocab.items()}

  print(f'vocab size: {tokenizer.vocab_size}')
  print(f'max length: {tokenizer.model_max_length}')
  bos_encoded_input = tokenizer.encode('<|startoftext|>', add_special_tokens=False)
  eos_encoded_input = tokenizer.encode(f'{tokenizer.eos_token}', add_special_tokens=False)
  print(f'{bcolors.OKCYAN}special token (bos): {tokenizer.bos_token}: {bos_encoded_input}{bcolors.ENDC}')
  print(f'{bcolors.OKCYAN}special token (eos): {tokenizer.eos_token}: {eos_encoded_input}{bcolors.ENDC}')
  print(f'{bcolors.OKGREEN}GREEN = Token within the 77-Token Limit{bcolors.ENDC}')
  print(f'{bcolors.RED}RED = Token outside 77-Token Limit{bcolors.ENDC}\n')

  #inputs = tokenizer(sys.argv[1], padding=True)
  inputs = tokenizer(f'{prompt_or_caption}', padding=True)


  print(textwrap.fill(prompt_or_caption, width=100))

  #  print the token ids in rows of 10
  row_length = 10
  final_string_ids = '' # final string for the ids
  final_string_words = '' # final string for the words/subwords

  # Find the length of the longest word (for padding)
  max_word_length = max(len(vocab[id]) for id in inputs["input_ids"])

  for i in range(0, len(inputs["input_ids"]), row_length):
    # Extract a row's worth of ids
    row = inputs["input_ids"][i:i + row_length]

    #  inputs["input_ids"][x] = the ID Numbers
    #  vocab[inputs["input_ids"][x]] = the sub-word

    # Initialize an empty string for the new row
    row_string_ids = ""
    row_string_words = ""

    # Iterate through each token-id in the row, using j as index
    for j, id in enumerate(row):
        # here, id = inputs["input_ids"][j]
        # Calculate the global index of the token
        global_index = i + j

        if show_end_of_word_marker:
          word = vocab[id]
        else:
          word = vocab[id].replace('</w>', '')  # Strip end-of-word marker

        padded_word = word.ljust(max_word_length)

        # Determine the color based on the global token index
        if global_index < 77:
            color = bcolors.OKGREEN
        else:
            color = bcolors.RED

        # Append the colored token to the row string
        row_string_ids += f"{color}{id:5d}{bcolors.ENDC}"
        row_string_words += f"{color}{padded_word}{bcolors.ENDC}"

        if j < len(row) - 1:
            row_string_ids += ", "  # Add comma and space between tokens
            row_string_words += ", "  # Add comma and space between tokens


    # Determine if a comma should be added at the end
    end_comma = ',' if i + row_length < len(inputs["input_ids"]) else ''

    # add to final strings
    final_string_ids += row_string_ids + end_comma + '\n'
    final_string_words  += row_string_words + end_comma + '\n'

  print('\n')
  token_count = global_index+1
  if token_count <= 77:
    print(f'Token Count: {bcolors.OKGREEN}{token_count}{bcolors.ENDC}')
  else:
    print(f'Token Count: {bcolors.RED}{token_count}{bcolors.ENDC}')

  print('\n')

  print(final_string_ids)

  print('\n')

  print(final_string_words)




In [ ]:
#@title 5. Run

show_end_of_word_marker = False

PROMPT_CAPTION = "cinematic photo film noir style, woman, highly detailed, sharp focus, ultra sharpness, monochrome, high contrast, dramatic shadows, 1940s style, mysterious, cinematic, hyperreal, hyperrealistic, hyperrealism, hyperrealistic painting, mkdrgs style, (b&w, Monochromatic, Film Photography:1.3), Photorealistic, Hyperrealistic, Hyperdetailed, film noir, analog style, soft lighting, subsurface scattering, realistic, heavy shadow, masterpiece, best quality, ultra realistic, 8k, golden ratio, Intricate, High Detail, film photography, soft focus, dramatic lighting, 35mm photograph, film, bokeh, professional, 4k, highly detailed" # @param {type:"string"}

tokenize(PROMPT_CAPTION, show_end_of_word_marker)

